# Building Web App

In [1]:
import datetime
import json

import requests
from flask import Flask
from flask import render_template, redirect, request

app = Flask(__name__)

In [2]:
posts = []
CONNECTED_NODE_ADDRESS=""

In [4]:
@app.route('/')
def index():
    return render_template('connect.html', title='Connect Node')



@app.route('/main_page', methods=['GET','POST'])
def main_page():
    if request.method == 'POST':
        global CONNECTED_NODE_ADDRESS
        CONNECTED_NODE_ADDRESS="""http://"""+request.form.get('node_address')
        
    return render_template('index.html',
                           title='YourNet: Decentralized '
                                 'content sharing',
                           posts=posts,
                           node_address=CONNECTED_NODE_ADDRESS,
                           readable_time=timestamp_to_string)    
    
    
@app.route('/fetch_chain', methods=['GET'])
def fetch_chain():
    get_chain_address = "{}/chain".format(CONNECTED_NODE_ADDRESS)
    response = requests.get(get_chain_address)
    if response.status_code == 200:
        content = []
        chain = json.loads(response.content)
        
        for blok in chain["chain"]:
            if blok["transactions"]:
                for t in blok["transactions"]:
                    data = {"author" : t["author"],
                            "content" : t["content"], 
                            "timestamp" : t["timestamp"]}


                    content.append(data)
                

        global posts
        posts = sorted(content,
                       key=lambda k: k['timestamp'],
                       reverse=True)
    return redirect('/main_page')


@app.route('/fetch_pending', methods=['GET'])
def fetch_pending():
    get_chain_address = "{}/pending_transactions".format(CONNECTED_NODE_ADDRESS)
    response = requests.get(get_chain_address)
    if response.status_code == 200:
        content = []
        pending = json.loads(response.content)

        for transaction in pending:
            data = {
                'author': transaction['author'],
                'content': transaction['content'],
                'timestamp': transaction['timestamp']
            }
            
            content.append(data)

        global posts
        posts = sorted(content,
                       key=lambda k: k['timestamp'],
                       reverse=True)
    return redirect('/main_page')


@app.route('/submit_transaction', methods=['POST'])
def submit_textarea():
    post_content = request.form["content"]
    author = request.form["author"]

    post_object = {
        'author': author,
        'content': post_content,
    }
    
    url = CONNECTED_NODE_ADDRESS + "/new_transaction"
    
    response = requests.post(url, json = post_object)
    #response_data = response.json()

    return redirect('/main_page')


def timestamp_to_string(epoch_time):
    return datetime.datetime.fromtimestamp(epoch_time).strftime('%H:%M')